In [9]:
# !pip install python-dotenv

In [10]:
from dotenv import dotenv_values
import os

ENV_PATH = '.env'

class Env_Path_Error(Exception):
    def __init__(self, path :str ):
        message = f"{path} does not exist"

        super().__init__(message)


def get_env(path):
    if not os.path.exists(ENV_PATH):
        raise Env_Path_Error(path = ENV_PATH)

    return dotenv_values(ENV_PATH)

env = get_env(ENV_PATH)

env['DOMO_INSTANCE'], env['DOMO_USERNAME']

('domo-community', 'jae@onyxreporting.com')

# AUTHENTICATION FUNCTION

In [11]:
import requests
from pprint import pprint

class GetAuthToken_Error(Exception):
    def __init__(self, domo_instance, domo_username):
        message = f"unable to retrieve an access_token for {domo_username} at {domo_instance}"
        super().__init__(message)

def get_auth_token(domo_username: str,
                   domo_password: str,
                   domo_instance: str,
                   debug_api: bool = False,
                   return_raw: bool = False
                   ):

    url = f"https://{domo_instance}.domo.com/api/content/v2/authentication"

    headers = {"Content-Type": "application/json"}

    body = {
        "method": "password",
        "emailAddress": domo_username,
        "password": domo_password
    }

    if debug_api:
        pprint({'url': url,
               'body': body
               })

    res = requests.request(url=url, method='POST', json=body, headers=headers)

    if return_raw:
        return res

    data = res.json()

    if not data.get('sessionToken'):
        raise GetAuthToken_Error(domo_instance=  domo_instance, domo_username = domo_username)
    
    # what happens if invalid authentication?
    
    return data['sessionToken']


session_token = get_auth_token(domo_username= env['DOMO_USERNAME'],
               domo_password= env['DOMO_PASSWORD'],
               domo_instance= env['DOMO_INSTANCE'],
               debug_api = False,
            #    return_raw = True
               )



# CLIENT FUNCTION

In [12]:
def get_data(session_token: str,
             url: str,
             method: str = 'GET',
             body: dict = None,
             additional_headers: dict = None,
             debug_api: bool = False,
             ):
    """run a request against Domo"""

    headers = {
        'x-domo-authentication': session_token,
        'Content-Type': 'application/json',
        'Accept': 'application/json'
    }

    if additional_headers:
        headers.update(**additional_headers)

    if debug_api:
        pprint({'url': url,
               'body': body,
               'headers' : headers
               })        

    res = requests.request(method=method,
                                url=url,
                                headers=headers,
                                json=body
                                )
    
    return res



# DATASET WRANGE ROUTES

In [13]:
def get_dataset_wrangle(session_token, domo_instance, dataset_id, 
debug_api: bool = False, return_raw: bool = False):

    url = f'https://{domo_instance}.domo.com/api/query/v1/datasources/{dataset_id}/wrangle'

    res = get_data(session_token=session_token,
                   url=url,
                   method='GET',
                   debug_api=debug_api)
    
    # what happens if dataset not found?
    if not res.status_code == 200: 
        raise Exception('not 200')

    if return_raw: 
        return res.json()
    
    data = res.json()
    return data['columns'] 


DATASET_ID = '51260f1d-b146-4dff-a1d8-68b7aa0b740b'

get_dataset_wrangle(session_token=session_token,
                    domo_instance=env['DOMO_INSTANCE'],
                    dataset_id=DATASET_ID,
                    debug_api = False)


[{'name': 'Order',
  'newName': 'Order',
  'description': '',
  'tags': None,
  'visible': True,
  'remove': False,
  'type': 'LONG',
  'formula': None,
  'calculatedExpr': None,
  'calculated': False,
  'order': 0},
 {'name': 'Logo',
  'newName': 'Logo',
  'description': 'hello world did this work? ',
  'tags': None,
  'visible': True,
  'remove': False,
  'type': 'STRING',
  'formula': None,
  'calculatedExpr': None,
  'calculated': False,
  'order': 1},
 {'name': 'Text',
  'newName': 'Text',
  'description': '',
  'tags': None,
  'visible': True,
  'remove': False,
  'type': 'STRING',
  'formula': None,
  'calculatedExpr': None,
  'calculated': False,
  'order': 2},
 {'name': 'URL',
  'newName': 'URL',
  'description': '',
  'tags': None,
  'visible': True,
  'remove': False,
  'type': 'STRING',
  'formula': None,
  'calculatedExpr': None,
  'calculated': False,
  'order': 3},
 {'name': 'Page Name',
  'newName': 'Page Name',
  'description': '',
  'tags': None,
  'visible': True,
  

In [14]:
def update_dataset_wrangle(session_token, domo_instance, dataset_id, column_ls: [dict],
debug_api: bool = False, return_raw: bool = False):

    url = f'https://{domo_instance}.domo.com/api/query/v1/datasources/{dataset_id}/wrangle'

    body = {'columns' : column_ls}

    res = get_data(session_token=session_token,
                   url=url,
                   method='POST',
                   body = body, 
                   debug_api=debug_api)
    
    # what happens if dataset not found?
    if not res.status_code == 200: 
        raise Exception('not 200')
        
    if return_raw: 
        return res
    
    return True

# IMPLEMENTATION
recommended to keep implementation separate from routes.  Routes should be stand alone functions that just do one task.

In [15]:
# original code

def update_column_description(session_token, dataset_id, column_name, attribute, value):
    url = f'https://{domo_instance}.domo.com/api/query/v1/datasources/{dataset_id}/wrangle'
    headers = {
        'x-domo-authentication': session_token,
        'Content-Type': 'application/json',
        'Accept': 'application/json'
    }
    response = requests.request("GET", url, headers=headers)
    json_resp = response.json()
    i = 0
    for key in json_resp['columns']:
    #loop through json, find the name attribute so we can update that specific matching column
    #Once found, update the json with a new value and POST it.
        for val in key:
            if (val == 'name') and (key[val] == column_name):
                json_resp['columns'][i][attribute] = value
            i += 1
    return requests.request("POST", url, headers=headers,json=json_resp).json()

In [16]:
def update_column_description(session_token, domo_instance,
                              dataset_id,
                              column_name,
                              attribute, new_value, debug_api: bool = False):
    url = f'https://{domo_instance}.domo.com/api/query/v1/datasources/{dataset_id}/wrangle'

    column_ls = get_dataset_wrangle(session_token=session_token,
                                    domo_instance=domo_instance,
                                    dataset_id=dataset_id,
                                    debug_api=debug_api)

    for column_obj in column_ls:
        # loop through json, find the name attribute so we can update that specific matching column
        # Once found, update the json with a new value and POST it.
        if column_obj['name'] == column_name:
            column_obj.update({attribute: new_value})

    return update_dataset_wrangle(session_token=session_token,
                                    domo_instance=domo_instance,
                                    dataset_id=dataset_id,
                                    column_ls = column_ls,
                                    debug_api=debug_api)

update_column_description(session_token=session_token,
                          domo_instance=env['DOMO_INSTANCE'],
                          dataset_id=DATASET_ID,
                          column_name='Logo',
                          attribute='description',
                          new_value='hello world did this work? ', debug_api=False)


True